In [2]:
from datasets import load_dataset

# Verisetini indirme
dataset = load_dataset("sajjadhadi/disease-diagnosis-dataset")

# Verinin örneklerine göz atma
print(dataset)

# Eğitim setinin ilk örneğine bakma
print(dataset['train'][0])


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iremm\.cache\huggingface\hub\datasets--sajjadhadi--disease-diagnosis-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 24695/24695 [00:00<00:00, 1643239.85 examples/s]


DatasetDict({
    train: Dataset({
        features: ['diagnosis', 'text'],
        num_rows: 469195
    })
    test: Dataset({
        features: ['diagnosis', 'text'],
        num_rows: 24695
    })
})
{'diagnosis': 'personality disorder', 'text': 'Patient reported these symptoms: depression ,hostile behavior ,drug abuse ,delusions or hallucinations ,temper problems ,fears and phobias and low self-esteem  may indicate. based on these symptoms, what disease may the patient have? The patient may have personality disorder.'}


In [4]:
import re
# Metin temizleme fonksiyonu
def clean_text(text):
    # Gereksiz başlangıç kısmını ve son kısmını temizle
    text = re.sub(r"Patient reported these symptoms: ", "", text)
    text = re.sub(r" based on these symptoms, what disease may the patient have\? The patient may have .+", "", text)
    return text.strip()

# Eğitim verisindeki 'text' bölümünü temizleme
dataset['train'] = dataset['train'].map(lambda x: {'text': clean_text(x['text']), 'diagnosis': x['diagnosis']})

# Test verisindeki 'text' bölümünü temizleme
dataset['test'] = dataset['test'].map(lambda x: {'text': clean_text(x['text']), 'diagnosis': x['diagnosis']})

# Temizlenmiş veriyi kontrol etme
print(dataset['train'][0])

Map: 100%|██████████| 24695/24695 [00:01<00:00, 21249.69 examples/s]

{'diagnosis': 'personality disorder', 'text': 'depression ,hostile behavior ,drug abuse ,delusions or hallucinations ,temper problems ,fears and phobias and low self-esteem  may indicate.'}


In [7]:
from transformers import MarianMTModel, MarianTokenizer

# Model ve tokenizer'ı yükleme
model_name = 'Helsinki-NLP/opus-mt-en-trk'  # İngilizce'den Türkçe'ye çeviri modeli
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Çeviri fonksiyonu
def translate(text):
    # Metni tokenlara çevirme
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Modeli kullanarak çeviri yapma
    translated = model.generate(**tokens)
    # Çevrilen metni çözme
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Eğitim verisindeki 'text' bölümünü Türkçeye çevirme
dataset['train'] = dataset['train'].map(lambda x: {'text': translate(x['text']), 'diagnosis': x['diagnosis']})

# Test verisindeki 'text' bölümünü Türkçeye çevirme
dataset['test'] = dataset['test'].map(lambda x: {'text': translate(x['text']), 'diagnosis': x['diagnosis']})

# Çevrilmiş veriyi kontrol etme
print(dataset['train'][0])


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iremm\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-trk. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ImportError: 
MarianTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
